#### Upgrade Propensity Model

##### Problem Statement:

In this task, we ask you to do an analysis to predict what customers will be most likely to upgrade
We provide a small sample of data you build the propensity model with.


In [271]:
# importing the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import warnings
warnings.filterwarnings('ignore')


##### Data Exploration

In [272]:
# Reading the two data sets provided for the analysis

df_data = pd.read_excel("DATA.xlsx",engine='openpyxl')
df_upgrade = pd.read_excel("UPGRADES.xlsx",engine='openpyxl')

In [273]:
df_data.head(3)


,SUBSCRIPTION_KEY,AVERAGE_SPEND,VOICE_OFFNET_CNT_L3M,VOICE_OFFNET_DUR_L3M,VOICE_OUTBOUND_ROAM_CNT_L3M,SMS_OFFNET_CNT_L3M,SMS_OUTBOUND_ROAM_CNT_L3M,DATA_CNT_L3M,DATA_VOLUME_L3M,NUM_OF_UPGRADES,HANDSET_FIRST_DATE,HANDSET_LATEST_DATE,LOYALTY_FLAG,SEGMENT,TOPUP_VALUE_L3M,TOPUP_CNT_L3M,CALLS_CARE_CNTS_L6M,AUTO_TOPUP
0,462713,2.71,6,283,0.0,2,0.0,0.0,0.0,2.0,2015-09-26,2016-01-07,No,Segment3,8.1301,1.0,0.0,N
1,272924,16.26,3,34,1.0,13,9.0,0.0,0.0,0.0,2015-07-24,2016-01-07,Yes,Segment4,48.7806,3.0,1.0,N
2,319500,5.42,15,"2,477",0.0,36,0.0,0.0,0.0,0.0,2012-09-04,2016-01-07,No,Segment2,16.2602,1.0,0.0,N


In [274]:
df_upgrade.head(3)

,SUBSCRIPTION_KEY,UPGRADE_FLAG
0,462713,NO
1,272924,NO
2,319500,NO


In [275]:
print("shape of data data set is: ",df_data.shape)
print("shape of upgarde data set is: ",df_upgrade.shape)

shape of data data set is:  (10350, 18)
shape of upgarde data set is:  (10350, 2)


In [276]:
df = pd.merge(df_data, df_upgrade, how="left", on=["SUBSCRIPTION_KEY"])

In [277]:
print("Shape of merged data set is: ",df.shape)

Shape of merged data set is:  (10350, 19)


In [278]:
df.head(3)

,SUBSCRIPTION_KEY,AVERAGE_SPEND,VOICE_OFFNET_CNT_L3M,VOICE_OFFNET_DUR_L3M,VOICE_OUTBOUND_ROAM_CNT_L3M,SMS_OFFNET_CNT_L3M,SMS_OUTBOUND_ROAM_CNT_L3M,DATA_CNT_L3M,DATA_VOLUME_L3M,NUM_OF_UPGRADES,HANDSET_FIRST_DATE,HANDSET_LATEST_DATE,LOYALTY_FLAG,SEGMENT,TOPUP_VALUE_L3M,TOPUP_CNT_L3M,CALLS_CARE_CNTS_L6M,AUTO_TOPUP,UPGRADE_FLAG
0,462713,2.71,6,283,0.0,2,0.0,0.0,0.0,2.0,2015-09-26,2016-01-07,No,Segment3,8.1301,1.0,0.0,N,NO
1,272924,16.26,3,34,1.0,13,9.0,0.0,0.0,0.0,2015-07-24,2016-01-07,Yes,Segment4,48.7806,3.0,1.0,N,NO
2,319500,5.42,15,"2,477",0.0,36,0.0,0.0,0.0,0.0,2012-09-04,2016-01-07,No,Segment2,16.2602,1.0,0.0,N,NO


In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10350 entries, 0 to 10349
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   SUBSCRIPTION_KEY             10350 non-null  int64         
 1   AVERAGE_SPEND                10300 non-null  float64       
 2   VOICE_OFFNET_CNT_L3M         10350 non-null  int64         
 3   VOICE_OFFNET_DUR_L3M         10350 non-null  object        
 4   VOICE_OUTBOUND_ROAM_CNT_L3M  10300 non-null  float64       
 5   SMS_OFFNET_CNT_L3M           10350 non-null  object        
 6   SMS_OUTBOUND_ROAM_CNT_L3M    10300 non-null  float64       
 7   DATA_CNT_L3M                 10300 non-null  float64       
 8   DATA_VOLUME_L3M              10300 non-null  float64       
 9   NUM_OF_UPGRADES              10300 non-null  float64       
 10  HANDSET_FIRST_DATE           10350 non-null  datetime64[ns]
 11  HANDSET_LATEST_DATE          10350 non-nu

In [280]:
df.isnull().sum()

SUBSCRIPTION_KEY                0
AVERAGE_SPEND                  50
VOICE_OFFNET_CNT_L3M            0
VOICE_OFFNET_DUR_L3M            0
VOICE_OUTBOUND_ROAM_CNT_L3M    50
SMS_OFFNET_CNT_L3M              0
SMS_OUTBOUND_ROAM_CNT_L3M      50
DATA_CNT_L3M                   50
DATA_VOLUME_L3M                50
NUM_OF_UPGRADES                50
HANDSET_FIRST_DATE              0
HANDSET_LATEST_DATE             0
LOYALTY_FLAG                   50
SEGMENT                        50
TOPUP_VALUE_L3M                50
TOPUP_CNT_L3M                  50
CALLS_CARE_CNTS_L6M            50
AUTO_TOPUP                     50
UPGRADE_FLAG                    0
dtype: int64

In [281]:
# checking for duplicates in the data set
print('Total number of duplicate values in the data set is/are: {}'.format(df.duplicated().sum()))

Total number of duplicate values in the data set is/are: 0


In [282]:
#checking for imbalance in the data set
df['UPGRADE_FLAG'].value_counts()

NO     10000
YES      350
Name: UPGRADE_FLAG, dtype: int64

##### Data Cleaning

In [283]:
# # SMS_OFFNET_CNT_L3M , VOICE_OFFNET_DUR_L3M , HANDSET_FIRST_DATE , HANDSET_LATEST_DATE , LOYALTY_FLAG , 
# # SEGMENT , AUTO_TOPUP , UPGRADE_FLAG

# df.HANDSET_FIRST_DATE.unique()

In [284]:
# df.AUTO_TOPUP.value_counts()

In [285]:
# removing the white spaces before the numbers for column SMS_OFFNET_CNT_L3M
df.SMS_OFFNET_CNT_L3M = df.SMS_OFFNET_CNT_L3M.str.strip()

In [286]:
# converting the type of the columns from object to int
df.SMS_OFFNET_CNT_L3M = df.SMS_OFFNET_CNT_L3M.str.replace(",","",regex=True) # removed , from the numbers that is 2,123 will be 2123 now
df.SMS_OFFNET_CNT_L3M = df.SMS_OFFNET_CNT_L3M.apply(pd.to_numeric) # converting it to numeric field
# df.SMS_OFFNET_CNT_L3M.unique()

In [287]:
# removing the white spaces before the numbers for column VOICE_OFFNET_DUR_L3M
df.VOICE_OFFNET_DUR_L3M = df.VOICE_OFFNET_DUR_L3M.str.strip()

In [288]:
# converting the type of the columns from object to int
df.VOICE_OFFNET_DUR_L3M = df.VOICE_OFFNET_DUR_L3M.str.replace(",","",regex=True) # removed , from the numbers that is 2,123 will be 2123 now
df.VOICE_OFFNET_DUR_L3M = df.VOICE_OFFNET_DUR_L3M.apply(pd.to_numeric) # converting it to numeric field
# df.VOICE_OFFNET_DUR_L3M.unique()

In [289]:
# replacing nan values of LOYALTY_FLAG field with No
df.LOYALTY_FLAG = df.LOYALTY_FLAG.replace(np.nan,"No",regex=True)

In [290]:
# converting yes and no to 0 and 1
df.LOYALTY_FLAG = df.LOYALTY_FLAG.replace({'No':0, 'Yes':1})

In [291]:
# replacing nan values of SEGMENT field with UNKNOWN
df.SEGMENT = df.SEGMENT.replace(np.nan,"UNKNOWN",regex=True)

In [292]:
# converting segments to numeric values
df.SEGMENT = df.SEGMENT.replace({'Segment1':1, 'Segment2':2,'Segment3':3,'Segment4':4,'UNKNOWN':5})

In [293]:
# replacing nan values of AUTO_TOPUP field with N
df.AUTO_TOPUP = df.AUTO_TOPUP.replace(np.nan,"N",regex=True)

In [294]:
# converting yes and no to 0 and 1
df.AUTO_TOPUP = df.AUTO_TOPUP.replace({'N':0, 'Y':1})

In [295]:
#converting No and yes to 0 and 1
df.UPGRADE_FLAG = df.UPGRADE_FLAG.replace({'NO':0, 'YES':1})

In [296]:
# droping two columns since it has only one value
df.drop(['DATA_CNT_L3M','DATA_VOLUME_L3M'],inplace = True, axis = 1)

In [297]:
# mode imputation in some column for null values
col_mode = ['NUM_OF_UPGRADES','TOPUP_CNT_L3M','CALLS_CARE_CNTS_L6M']

for i in col_mode:
    df[i]=  df[i].fillna((df[i].mode()[0]))

In [298]:
# mean imputation in some column for null values
col_mean = ['AVERAGE_SPEND','VOICE_OUTBOUND_ROAM_CNT_L3M','SMS_OUTBOUND_ROAM_CNT_L3M','TOPUP_VALUE_L3M']

for i in col_mean:
    df[i].fillna((df[i].mean()),inplace = True)

In [299]:
df.dtypes

SUBSCRIPTION_KEY                        int64
AVERAGE_SPEND                         float64
VOICE_OFFNET_CNT_L3M                    int64
VOICE_OFFNET_DUR_L3M                    int64
VOICE_OUTBOUND_ROAM_CNT_L3M           float64
SMS_OFFNET_CNT_L3M                      int64
SMS_OUTBOUND_ROAM_CNT_L3M             float64
NUM_OF_UPGRADES                       float64
HANDSET_FIRST_DATE             datetime64[ns]
HANDSET_LATEST_DATE            datetime64[ns]
LOYALTY_FLAG                            int64
SEGMENT                                 int64
TOPUP_VALUE_L3M                       float64
TOPUP_CNT_L3M                         float64
CALLS_CARE_CNTS_L6M                   float64
AUTO_TOPUP                              int64
UPGRADE_FLAG                            int64
dtype: object

In [300]:
df.isnull().sum()

SUBSCRIPTION_KEY               0
AVERAGE_SPEND                  0
VOICE_OFFNET_CNT_L3M           0
VOICE_OFFNET_DUR_L3M           0
VOICE_OUTBOUND_ROAM_CNT_L3M    0
SMS_OFFNET_CNT_L3M             0
SMS_OUTBOUND_ROAM_CNT_L3M      0
NUM_OF_UPGRADES                0
HANDSET_FIRST_DATE             0
HANDSET_LATEST_DATE            0
LOYALTY_FLAG                   0
SEGMENT                        0
TOPUP_VALUE_L3M                0
TOPUP_CNT_L3M                  0
CALLS_CARE_CNTS_L6M            0
AUTO_TOPUP                     0
UPGRADE_FLAG                   0
dtype: int64

In [301]:
df.shape

(10350, 17)

##### Feature Engineering

In [302]:
df['Number_of_Days'] = (df['HANDSET_LATEST_DATE'] - df['HANDSET_FIRST_DATE']).dt.days

In [303]:
df.drop(['HANDSET_LATEST_DATE','HANDSET_FIRST_DATE'],inplace =True,axis = 1)

In [304]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SUBSCRIPTION_KEY,10350.0,238842.335362,137379.999580,4.0000,120194.2500,238182.0000,357392.2500,477531.0000
AVERAGE_SPEND,10350.0,10.210563,5.464593,1.3500,5.4200,10.8400,16.2600,46.0700
VOICE_OFFNET_CNT_L3M,10350.0,14.447729,21.641523,0.0000,0.0000,5.0000,19.0000,204.0000
VOICE_OFFNET_DUR_L3M,10350.0,896.627343,1478.099922,0.0000,0.0000,228.5000,1161.0000,18873.0000
VOICE_OUTBOUND_ROAM_CNT_L3M,10350.0,1.683107,7.258700,0.0000,0.0000,0.0000,0.0000,380.0000
SMS_OFFNET_CNT_L3M,10350.0,63.911014,205.481564,0.0000,0.0000,9.0000,50.0000,6538.0000
SMS_OUTBOUND_ROAM_CNT_L3M,10350.0,2.631942,16.101937,0.0000,0.0000,0.0000,0.0000,681.0000
NUM_OF_UPGRADES,10350.0,0.071594,0.290266,0.0000,0.0000,0.0000,0.0000,3.0000
LOYALTY_FLAG,10350.0,0.323478,0.467826,0.0000,0.0000,0.0000,1.0000,1.0000
SEGMENT,10350.0,2.573237,1.014628,1.0000,2.0000,3.0000,3.0000,5.0000
